In [1]:
import numpy as np
import pandas as pd

class Multinomial_NB:
    
    def __init__(self, alpha=1.0):
        self.alpha = alpha
    
    def fit(self, X, Y):
        labels, label_counts = np.unique(Y, return_counts=True)
        self.labels = labels
        self.label_counts = label_counts
        pi_c = np.array([])
        theta_jc = dict()
        
        for c in range(len(labels)):
            temp = (self.alpha + label_counts[c])/((self.alpha * len(labels)) + len(Y))
            pi_c = np.append(pi_c, temp)
        
        classes = np.array([])
        for c in range(len(labels)):
            class_dict = dict(name=labels[c], words=np.array([]))
        
        words_dict = dict()
        total_word_count = dict()
        for i in range(len(X)):
            words, word_counts = np.unique(X[i], return_counts=True)
            current_class = Y[i]
            
            if current_class in total_word_count:
                total_word_count[current_class] += len(X.shape[1])
            else:
                total_word_count[current_class] = len(X.shape[1])
            
            for j in range(len(words)):
                if words[j] in words_dict:
                    words_dict[words[j]][current_class] += word_counts[j]
                else:
                    words_dict[words[j]] = dict()
                    words_dict[words[j]][current_class] = word_counts[j]
                    
        for j in len(words_dict):
            words_dict_keys = words_dict.keys()
            word_classes_dict = dict()
            
            for c in len(words_dict[words_dict_keys[j]]):
                classes_dict_keys = words_dict[words_dict_keys[j]].keys()
                temp = (self.alpha + words_dict[words_dict_keys[j]][classes_dict_keys[c]])/((self.alpha * X.shape[1]) + total_word_count[classes_dict_keys[c]])
                theta_jc[words_dict[j]] = dict()
                theta_jc[words_dict[j]][classes_dict_keys[c]] = temp
                 
        self.pi_c = pi_c
        self.theta_jc = theta_jc
    
    def predict(self, X):
        log_l = np.array([])
        for i in range(len(X)):
            words, word_counts = np.unique(X[i], return_counts=True)
            row_class_probs = dict()
            for c in range(len(self.labels)):
                temp_log_pi = np.log(self.pi_c[c]**self.label_counts[c])
                
                temp_log_sum_theta = 0
                for j in range(X.shape[1]):
                    word_count = word_counts[words.index(X[i,j])]
                    temp = (self.theta_jc[X[i,j]]**word_count)*((1-self.theta_jc[X[i,j]])**(X.shape[1]-word_count))
                    temp_log_sum_theta += np.log(temp)
                
                row_class_probs[self.labels[c]] = temp_log_pi + temp_log_sum_theta
            
            class_chosen = row_class_probs.keys()[0]
            for key in row_class_probs:
                if row_class_probs[key] > row_class_probs[class_chosen]:
                    class_chosen = key

            log_l = np.append(log_l, class_chosen)
        
        return log_l
    
    def predict_proba(self, X):
        pass
    
    def predict_log_proba(self, X):
        pass

In [2]:
import pandas as pd
df = pd.read_csv("./datasets/SMSSpamCollection.csv",header=None,names=["label","text"])
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
df['text_lemmatized'] = df['text'].map(lambda text: ' '.join(lemmatizer.lemmatize(w) for w in nltk.word_tokenize(text)))
y=np.array(df['label'])
labels, counts = np.unique(y, return_counts=True)
dictionary = dict(zip(labels, counts))
print(counts)
print(dictionary[y[0]])
# print(dictionary[y[:]])
for i in range(len(y)):
    print(dictionary[y[i]])
# print(y[:])



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mattb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[4827  747]
4827
4827
4827
747
4827
4827
747
4827
4827
747
747
4827
747
747
4827
4827
747
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
747
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
747
4827
4827
747
747
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
747
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
747
4827
4827
4827
747
747
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4

4827
4827
4827
4827
4827
4827
4827
4827
747
4827
747
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
747
747
4827
4827
747
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
747
747
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
747
4827
747
4827
4827
4827
747
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
482

4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
747
747
4827
747
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
747
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
747
4827
747
4827
4827
747
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
747
4827
747
4827
4827
747
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
747
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
747
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
4827
482

In [67]:
def onehot(Y):  
    U_index = np.unique(Y, return_inverse=1)[1]
    Y_one_hot=(U_index.ravel()[:,None] == np.arange(U_index.max()+1)).astype(int)
    return Y_one_hot
def softmax(score):
    Y_proba=np.exp(score).T/(np.sum(np.exp(score),axis=1))
    return Y_proba.T
def cross_entropy_loss(Y_one_hot, Y_proba):
    ce_cost=-Y_one_hot*np.log2(Y_proba)
    return ce_cost[np.nonzero(ce_cost)].mean()

In [68]:
class Softmax_Regression:
    
    def __init__(self, alpha=1.0):
        self.alpha = alpha
    
    def fit(self, X, Y, learning_rate=0.01, epochs=1000, tol=None, regularizer=None,lambd=0.0,early_stopping=False, validation_fraction=0.1,**kwargs):
        if early_stopping == True:
            X, X_val, Y, Y_val=partition(X, Y, validation_fraction)
            accuracies=[]
            accuracy=0.0
        Y=onehot(Y)
        bias_term=np.ones((X.shape[0], 1))
        X=np.hstack((bias_term,X))
        X=np.asarray(X)
        theta=np.zeros((X.shape[1], Y.shape[1]))
        
        for i in range(epochs):
            scores=X.dot(theta)
            Y_proba=softmax(scores)
            ce=cross_entropy_loss(Y, Y_proba)
            if regularizer==None:
                theta=theta-(learning_rate/X.shape[0])*np.dot(X.T,(Y_proba-Y))
            elif regularizer=='l1':
                theta=theta-(learning_rate/X.shape[0]*np.dot(X.T,(Y_proba-Y))+lambd*np.sign(theta))
            else:
                theta=theta-(learning_rate/X.shape[0]*np.dot(X.T,(Y_proba-Y))+lambd*theta)
            new_scores=X.dot(theta)
            new_Y_proba=softmax(scores)
            new_ce=cross_entropy_loss(Y, Y_proba)
 
            if tol is not None:
                if new_ce > ce-tol:
                    break
            if early_stopping == True:
                X_val=np.asarray(X_val)
                bias_term=np.ones((X_val.shape[0], 1))
                X_val=np.hstack((bias_term,X_val))
                scores=X_val.dot(self.theta)
                Y_proba=softmax(scores)
                new_accuracy=accuracy(np.argmax(Y_proba,axis=1),y_val)
                accuracies.append(new_accuracy)
                if new_accuracy < accuracy:
                    break
                
        self.theta=theta
        if early_stopping == True:
            return accuracies
        
    def predict(self, X):
        X=np.asarray(X)
        bias_term=np.ones((X.shape[0], 1))
        X=np.hstack((bias_term,X))
        scores=X.dot(self.theta)
        Y_proba=softmax(scores)
        return np.argmax(Y_proba,axis=1)
    

In [69]:
def partition(feature_matrix, target_vector, t):
    last_index_taken = len(feature_matrix) - int(round(len(feature_matrix) * t))
    X_train = feature_matrix[0:last_index_taken,:]
    X_test = feature_matrix[last_index_taken:len(feature_matrix), :]
    y_train = target_vector[0:last_index_taken]
    y_test = target_vector[last_index_taken:len(target_vector)]
    return X_train, X_test, y_train, y_test

def accuracy(predicted, actual):
    accuracy = np.sum(predicted == actual)
    return accuracy/len(actual)
def kfold(folds, data, labels, model, model_args, score_function,seed=None):
    if (seed == None):
        seed = np.random.randint(1,500)
    data = np.asarray(data)
    #print(data.shape)
    labels = np.asarray(labels)
    #print(labels.shape)
    np.random.seed(seed)
    np.random.shuffle(data)
    np.random.seed(seed)
    np.random.shuffle(labels)
    
    datafolds = np.array_split(data,folds)
    labelfolds = np.array_split(labels,folds)
    
    scores = []
    true_labels = np.array([])
    predictions = np.array([])

    for fold in range(len(datafolds)):
        trainingdatafolds = np.vstack([data for k, data in enumerate(datafolds) if k not in [fold]])
        trainingdatafolds = trainingdatafolds.reshape(-1,data.shape[1])
        trainingdata = np.empty((0,data.shape[1]), int)
        trainingdata = np.asmatrix(np.append(trainingdata, trainingdatafolds, axis=0))
        
        traininglabelfolds = np.concatenate([label for k, label in enumerate(labelfolds) if k not in [fold]])
        traininglabelfolds = traininglabelfolds.reshape(-1,1)
        traininglabels = np.empty((0,1), int)
        traininglabels = np.append(traininglabels, traininglabelfolds, axis=0)
    
        testingdata = np.asmatrix(datafolds[fold])
        testinglabels = labelfolds[fold]
        
        calledmodel = model()
        calledmodel.fit(X=trainingdata, Y=traininglabels, kwargs=model_args)
        preds = calledmodel.predict(testingdata)
        score = score_function(preds,testinglabels)
        scores = np.append(scores,score)
        true_labels = np.append(labels,testinglabels)
        predictions = np.append(predictions,preds)
    
    
    results={'average_score':np.mean(scores),'predicted_values':predictions, 'true_values':true_labels}

    return results

def confusion_matrix(predicted, actual):
    cm = np.zeros((len(np.unique(actual)), len(np.unique(actual))))
    for i, j in zip(predicted, actual):
        cm[i][j] += 1
    return cm

In [70]:
from sklearn.datasets import load_iris
data = load_iris()
Y=data['target']
X=data['data']
np.random.seed(123)
Y= np.random.permutation(Y)

np.random.seed(123)
X= np.random.permutation(X)

X_train, X_test, Y_train, Y_test =partition(X, Y, 0.2)

In [ ]:
lambd_values= [1.0,0.1,0.01, 0.001, 0.0001]
tol_values=[None,0.001, 0.0001, 0.00001, 0.000001, 0.0000001]
learning_rate_values = [ 0.1, 0.01, 0.001]
regularizer_values= ['l1', 'l2']
value_obj = dict()
results_arr = np.array([])

for lambd in range(len(lambd_values)):
    for tol in range(len(tol_values)):
        for learning_rate in range(len(learning_rate_values)):
            for regularizer in range(len(regularizer_values)):
                hyperparameter_values={'lambd_values':lambd_values[lambd],'tol_values':tol_values[tol],'learning_rate':learning_rate_values[learning_rate],'regularizer':regularizer_values[regularizer]}
                results=kfold(5,X_train, Y_train, Softmax_Regression, hyperparameter_values,accuracy)
                value_obj['hyperparameter_values'] = hyperparameter_values
                value_obj = {
                    'hyperparameter_values': hyperparameter_values,
                    'average_score': results['average_score']
                }
                results_arr = np.append(results_arr, value_obj)
results_arr = sorted(results_arr, key=lambda k: k['average_score'])
results_arr[0]

In [ ]:
SR=Softmax_Regression()
SR.fit(X_train,Y_train,lambd=0.001,tol=None,learning_rate=0.01,regularizer='l2')
y_preds=SR.predict(X_test)
print(Y_test)
print(y_preds)
print(accuracy(Y_test,y_preds))
print(confusion_matrix(Y_test,y_preds))